# OLED SSD-1351

[Tutorial](https://www.rototron.info/raspberry-pi-esp32-micropython-oled-tutorial/)

[bought from AliExpress](https://www.aliexpress.com/item/1-5-inch-7PIN-Full-Color-OLED-module-Display-Screen-SSD1351-Drive-IC-128-RGB-128/32793875682.html?spm=2114.search0104.3.8.4cc06bc8tvj9BN&ws_ab_test=searchweb0_0,searchweb201602_3_10152_5722813_10151_10065_10344_10068_5722613_10342_5722913_10343_10340_10341_10696_10084_10083_5722713_10618_10307_10059_100031_10103_10624_10623_10622_10621_10620_5722513,searchweb201603_25,ppcSwitch_5&algo_expid=0a85d1c7-352c-4dbe-80ea-989caddbc793-1&algo_pvid=0a85d1c7-352c-4dbe-80ea-989caddbc793&priceBeautifyAB=0)

In [1]:
print(1)

1


In [2]:
import upip

I (26523) modsocket: Initializing


In [4]:
upip.install('micropython-ssd1351')

Installing to: /lib/
Error installing 'micropython-ssd1351': list index out of range, packages may be partially installed


In [2]:
"""SSD1351 OLED module."""
from utime import sleep_ms
from math import cos, sin, pi, radians


def color565(r, g, b):
    """Return RGB565 color value.

    Args:
        r (int): Red value.
        g (int): Green value.
        b (int): Blue value.
    """
    return (r & 0xf8) << 8 | (g & 0xfc) << 3 | b >> 3


class Display(object):
    """Serial interface for 16-bit color (5-6-5 RGB) SSD1351 OLED display.

    Note:  All coordinates are zero based.
    """

    # Command constants from SSD1351 datasheet
    SET_COLUMN = const(0x15)
    SET_ROW = const(0x75)
    WRITE_RAM = const(0x5C)
    READ_RAM = const(0x5D)
    SET_REMAP = const(0xA0)
    START_LINE = const(0xA1)
    DISPLAY_OFFSET = const(0xA2)
    DISPLAY_ALL_OFF = const(0xA4)
    DISPLAY_ALL_ON = const(0xA5)
    NORMAL_DISPLAY = const(0xA6)
    INVERT_DISPLAY = const(0xA7)
    FUNCTION_SELECT = const(0xAB)
    DISPLAY_OFF = const(0xAE)
    DISPLAY_ON = const(0xAF)
    PRECHARGE = const(0xB1)
    DISPLAY_ENHANCEMENT = const(0xB2)
    CLOCK_DIV = const(0xB3)
    SET_VSL = const(0xB4)
    SET_GPIO = const(0xB5)
    PRECHARGE2 = const(0xB6)
    SET_GRAY = const(0xB8)
    USE_LUT = const(0xB9)
    PRECHARGE_LEVEL = const(0xBB)
    VCOMH = const(0xBE)
    CONTRAST_ABC = const(0xC1)
    CONTRAST_MASTER = const(0xC7)
    MUX_RATIO = const(0xCA)
    COMMAND_LOCK = const(0xFD)
    HORIZ_SCROLL = const(0x96)
    STOP_SCROLL = const(0x9E)
    START_SCROLL = const(0x9F)

    def __init__(self, spi, cs, dc, rst, width=128, height=128):
        """Initialize OLED.

        Args:
            spi (Class Spi):  SPI interface for OLED
            cs (Class Pin):  Chip select pin
            dc (Class Pin):  Data/Command pin
            rst (Class Pin):  Reset pin
            width (Optional int): Screen width (default 128)
            height (Optional int): Screen height (default 128)
        """
        self.spi = spi
        self.cs = cs
        self.dc = dc
        self.rst = rst
        self.width = width
        self.height = height
        # Initialize GPIO pins
        self.cs.init(self.cs.OUT, value=1)
        self.dc.init(self.dc.OUT, value=0)
        self.rst.init(self.rst.OUT, value=1)
        self.reset()
        # Send initialization commands
        self.write_cmd(self.COMMAND_LOCK, 0x12)  # Unlock IC MCU interface
        self.write_cmd(self.COMMAND_LOCK, 0xB1)  # A2,B1,B3,BB,BE,C1
        self.write_cmd(self.DISPLAY_OFF)  # Display off
        self.write_cmd(self.DISPLAY_ENHANCEMENT, 0xA4, 0x00, 0x00)
        self.write_cmd(self.CLOCK_DIV, 0xF0)  # Clock divider F1 or F0
        self.write_cmd(self.MUX_RATIO, 0x7F)  # Mux ratio
        self.write_cmd(self.SET_REMAP, 0x74)  # Segment remapping
        self.write_cmd(self.START_LINE, 0x00)  # Set Display start line
        self.write_cmd(self.DISPLAY_OFFSET, 0x00)  # Set display offset
        self.write_cmd(self.SET_GPIO, 0x00)  # Set GPIO
        self.write_cmd(self.FUNCTION_SELECT, 0x01)  # Function select
        self.write_cmd(self.PRECHARGE, 0x32),  # Precharge
        self.write_cmd(self.PRECHARGE_LEVEL, 0x1F)  # Precharge level
        self.write_cmd(self.VCOMH, 0x05)  # Set VcomH voltage
        self.write_cmd(self.NORMAL_DISPLAY)  # Normal Display
        self.write_cmd(self.CONTRAST_MASTER, 0x0A)  # Contrast master
        self.write_cmd(self.CONTRAST_ABC, 0xFF, 0xFF, 0xFF)  # Contrast RGB
        self.write_cmd(self.SET_VSL, 0xA0, 0xB5, 0x55)  # Set segment low volt.
        self.write_cmd(self.PRECHARGE2, 0x01)  # Precharge2
        self.write_cmd(self.DISPLAY_ON)  # Display on
        self.clear()

    def block(self, x0, y0, x1, y1, data):
        """Write a block of data to display.

        Args:
            x0 (int):  Starting X position.
            y0 (int):  Starting Y position.
            x1 (int):  Ending X position.
            y1 (int):  Ending Y position.
            data (bytes): Data buffer to write.
        """
        self.write_cmd(self.SET_COLUMN, x0, x1)
        self.write_cmd(self.SET_ROW, y0, y1)
        self.write_cmd(self.WRITE_RAM)
        self.write_data(data)

    def cleanup(self):
        """Clean up resources."""
        self.clear()
        self.display_off()
        self.spi.deinit()
        print('display off')

    def clear(self, color=0):
        """Clear display.

        Args:
            color (Optional int): RGB565 color value (Default: 0 = Black).
        """
        w = self.width
        h = self.height
        # Clear display in 1024 byte blocks
        if color:
            line = color.to_bytes(2, 'big') * 1024
        else:
            line = bytearray(2048)
        for x in range(0, w, 8):
            self.block(x, 0, x + 7, h - 1, line)

    def contrast(self, level):
        """Set display contrast to specified level.

        Args:
            level (int): Contrast level (0 - 15).
        Note:
            Can pass list to specifiy
        """
        assert(0 <= level < 16)
        self.write_cmd(self.CONTRAST_MASTER, level)

    def display_off(self):
        """Turn display off."""
        self.write_cmd(self.DISPLAY_OFF)

    def display_on(self):
        """Turn display on."""
        self.write_cmd(self.DISPLAY_ON)

    def draw_circle(self, x0, y0, r, color):
        """Draw a circle.

        Args:
            x0 (int): X coordinate of center point.
            y0 (int): Y coordinate of center point.
            r (int): Radius.
            color (int): RGB565 color value.
        """
        f = 1 - r
        dx = 1
        dy = -r - r
        x = 0
        y = r
        self.draw_pixel(x0, y0 + r, color)
        self.draw_pixel(x0, y0 - r, color)
        self.draw_pixel(x0 + r, y0, color)
        self.draw_pixel(x0 - r, y0, color)
        while x < y:
            if f >= 0:
                y -= 1
                dy += 2
                f += dy
            x += 1
            dx += 2
            f += dx
            self.draw_pixel(x0 + x, y0 + y, color)
            self.draw_pixel(x0 - x, y0 + y, color)
            self.draw_pixel(x0 + x, y0 - y, color)
            self.draw_pixel(x0 - x, y0 - y, color)
            self.draw_pixel(x0 + y, y0 + x, color)
            self.draw_pixel(x0 - y, y0 + x, color)
            self.draw_pixel(x0 + y, y0 - x, color)
            self.draw_pixel(x0 - y, y0 - x, color)

    def draw_ellipse(self, x0, y0, a, b, color):
        """Draw an ellipse.

        Args:
            x0, y0 (int): Coordinates of center point.
            a (int): Semi axis horizontal.
            b (int): Semi axis vertical.
            color (int): RGB565 color value.
        Note:
            The center point is the center of the x0,y0 pixel.
            Since pixels are not divisible, the axes are integer rounded
            up to complete on a full pixel.  Therefore the major and
            minor axes are increased by 1.
        """
        a2 = a * a
        b2 = b * b
        twoa2 = a2 + a2
        twob2 = b2 + b2
        x = 0
        y = b
        px = 0
        py = twoa2 * y
        # Plot initial points
        self.draw_pixel(x0 + x, y0 + y, color)
        self.draw_pixel(x0 - x, y0 + y, color)
        self.draw_pixel(x0 + x, y0 - y, color)
        self.draw_pixel(x0 - x, y0 - y, color)
        # Region 1
        p = round(b2 - (a2 * b) + (0.25 * a2))
        while px < py:
            x += 1
            px += twob2
            if p < 0:
                p += b2 + px
            else:
                y -= 1
                py -= twoa2
                p += b2 + px - py
            self.draw_pixel(x0 + x, y0 + y, color)
            self.draw_pixel(x0 - x, y0 + y, color)
            self.draw_pixel(x0 + x, y0 - y, color)
            self.draw_pixel(x0 - x, y0 - y, color)
        # Region 2
        p = round(b2 * (x + 0.5) * (x + 0.5) +
                  a2 * (y - 1) * (y - 1) - a2 * b2)
        while y > 0:
            y -= 1
            py -= twoa2
            if p > 0:
                p += a2 - py
            else:
                x += 1
                px += twob2
                p += a2 - py + px
            self.draw_pixel(x0 + x, y0 + y, color)
            self.draw_pixel(x0 - x, y0 + y, color)
            self.draw_pixel(x0 + x, y0 - y, color)
            self.draw_pixel(x0 - x, y0 - y, color)

    def draw_hline(self, x, y, w, color):
        """Draw a horizontal line.

        Args:
            x (int): Starting X position.
            y (int): Starting Y position.
            w (int): Width of line.
            color (int): RGB565 color value.
        """
        if self.is_off_grid(x, y, x + w - 1, y):
            return
        line = color.to_bytes(2, 'big') * w
        self.block(x, y, x + w - 1, y, line)

    def draw_image(self, path, x=0, y=0, w=128, h=128):
        """Draw image from flash.

        Args:
            path (string): Image file path.
            x (int): X coordinate of image left.  Default is 0.
            y (int): Y coordinate of image top.  Default is 0.
            w (int): Width of image.  Default is 128.
            h (int): Height of image.  Default is 128.
        """
        x2 = x + w - 1
        y2 = y + h - 1
        if self.is_off_grid(x, y, x2, y2):
            return
        with open(path, "rb") as f:
            chunk_height = 1024 // w
            chunk_count, remainder = divmod(h, chunk_height)
            chunk_size = chunk_height * w * 2
            chunk_y = y
            if chunk_count:
                for c in range(0, chunk_count):
                    buf = f.read(chunk_size)
                    self.block(x, chunk_y,
                               x2, chunk_y + chunk_height - 1,
                               buf)
                    chunk_y += chunk_height
            if remainder:
                buf = f.read(remainder * w * 2)
                self.block(x, chunk_y,
                           x2, chunk_y + remainder - 1,
                           buf)

    def draw_letter(self, x, y, letter, font, color, background=0,
                    landscape=False):
        """Draw a letter.

        Args:
            x (int): Starting X position.
            y (int): Starting Y position.
            letter (string): Letter to draw.
            font (XglcdFont object): Font.
            color (int): RGB565 color value.
            background (int): RGB565 background color (default: black).
            landscape (bool): Orientation (default: False = portrait)
        """
        buf, w, h = font.get_letter(letter, color, background,
                                    landscape)
        # Check for errors
        if w == 0:
            return w, h

        if landscape:
            y -= w
            if self.is_off_grid(x, y, x + h - 1, y + w - 1):
                return
            self.block(x, y,
                       x + h - 1, y + w - 1,
                       buf)
        else:
            if self.is_off_grid(x, y, x + w - 1, y + h - 1):
                return
            self.write_cmd(self.SET_REMAP, 0x75)  # Vertical address increment
            self.block(x, y,
                       x + w - 1, y + h - 1,
                       buf)
            self.write_cmd(self.SET_REMAP, 0x74)  # Switch back to horizontal
        return w, h

    def draw_line(self, x1, y1, x2, y2, color):
        """Draw a line using Bresenham's algorithm.

        Args:
            x1, y1 (int): Starting coordinates of the line
            x2, y2 (int): Ending coordinates of the line
            color (int): RGB565 color value.
        """
        # Check for horizontal line
        if y1 == y2:
            if x1 > x2:
                x1, x2 = x2, x1
            self.draw_hline(x1, y1, x2 - x1 + 1, color)
            return
        # Check for vertical line
        if x1 == x2:
            if y1 > y2:
                y1, y2 = y2, y1
            self.draw_vline(x1, y1, y2 - y1 + 1, color)
            return
        # Confirm coordinates in boundary
        if self.is_off_grid(min(x1, x2), min(y1, y2),
                            max(x1, x2), max(y1, y2)):
            return
        # Changes in x, y
        dx = x2 - x1
        dy = y2 - y1
        # Determine how steep the line is
        is_steep = abs(dy) > abs(dx)
        # Rotate line
        if is_steep:
            x1, y1 = y1, x1
            x2, y2 = y2, x2
        # Swap start and end points if necessary
        if x1 > x2:
            x1, x2 = x2, x1
            y1, y2 = y2, y1
        # Recalculate differentials
        dx = x2 - x1
        dy = y2 - y1
        # Calculate error
        error = dx >> 1
        ystep = 1 if y1 < y2 else -1
        y = y1
        for x in range(x1, x2 + 1):
            # Had to reverse HW ????
            if not is_steep:
                self.draw_pixel(x, y, color)
            else:
                self.draw_pixel(y, x, color)
            error -= abs(dy)
            if error < 0:
                y += ystep
                error += dx

    def draw_lines(self, coords, color):
        """Draw multiple lines.

        Args:
            coords ([[int, int],...]): Line coordinate X, Y pairs
            color (int): RGB565 color value.
        """
        # Starting point
        x1, y1 = coords[0]
        # Iterate through coordinates
        for i in range(1, len(coords)):
            x2, y2 = coords[i]
            self.draw_line(x1, y1, x2, y2, color)
            x1, y1 = x2, y2

    def draw_pixel(self, x, y, color):
        """Draw a single pixel.

        Args:
            x (int): X position.
            y (int): Y position.
            color (int): RGB565 color value.
        """
        if self.is_off_grid(x, y, x, y):
            return
        self.block(x, y, x, y, color.to_bytes(2, 'big'))

    def draw_polygon(self, sides, x0, y0, r, color, rotate=0):
        """Draw an n-sided regular polygon.

        Args:
            sides (int): Number of polygon sides.
            x0, y0 (int): Coordinates of center point.
            r (int): Radius.
            color (int): RGB565 color value.
            rotate (Optional float): Rotation in degrees relative to origin.
        Note:
            The center point is the center of the x0,y0 pixel.
            Since pixels are not divisible, the radius is integer rounded
            up to complete on a full pixel.  Therefore diameter = 2 x r + 1.
        """
        coords = []
        theta = radians(rotate)
        n = sides + 1
        for s in range(n):
            t = 2.0 * pi * s / sides + theta
            coords.append([int(r * cos(t) + x0), int(r * sin(t) + y0)])

        # Cast to python float first to fix rounding errors
        self.draw_lines(coords, color=color)

    def draw_rectangle(self, x, y, w, h, color):
        """Draw a rectangle.

        Args:
            x (int): Starting X position.
            y (int): Starting Y position.
            w (int): Width of rectangle.
            h (int): Height of rectangle.
            color (int): RGB565 color value.
        """
        x2 = x + w - 1
        y2 = y + h - 1
        self.draw_hline(x, y, w, color)
        self.draw_hline(x, y2, w, color)
        self.draw_vline(x, y, h, color)
        self.draw_vline(x2, y, h, color)

    def draw_sprite(self, buf, x, y, w, h):
        """Draw a sprite (optimized for horizontal drawing).

        Args:
            buf (bytearray): Buffer to draw.
            x (int): Starting X position.
            y (int): Starting Y position.
            w (int): Width of drawing.
            h (int): Height of drawing.
        """
        x2 = x + w - 1
        y2 = y + h - 1
        if self.is_off_grid(x, y, x2, y2):
            return
        self.block(x, y, x2, y2, buf)

    def draw_text(self, x, y, text, font, color,  background=0,
                  landscape=False, spacing=1):
        """Draw text.

        Args:
            x (int): Starting X position.
            y (int): Starting Y position.
            text (string): Text to draw.
            font (XglcdFont object): Font.
            color (int): RGB565 color value.
            background (int): RGB565 background color (default: black).
            landscape (bool): Orientation (default: False = portrait)
            spacing (int): Pixels between letters (default: 1)
        """
        for letter in text:
            # Get letter array and letter dimensions
            w, h = self.draw_letter(x, y, letter, font, color, background,
                                    landscape)
            # Stop on error
            if w == 0 or h == 0:
                return

            if landscape:
                # Fill in spacing
                if spacing:
                    self.fill_hrect(x, y - w - spacing, h, spacing, background)
                # Position y for next letter
                y -= (w + spacing)
            else:
                # Fill in spacing
                if spacing:
                    self.fill_vrect(x + w, y, spacing, h, background)
                # Position x for next letter
                x += w + spacing

    def draw_vline(self, x, y, h, color):
        """Draw a vertical line.

        Args:
            x (int): Starting X position.
            y (int): Starting Y position.
            h (int): Height of line.
            color (int): RGB565 color value.
        """
        # Confirm coordinates in boundary
        if self.is_off_grid(x, y, x, y + h):
            return
        line = color.to_bytes(2, 'big') * h
        self.block(x, y, x, y + h - 1, line)

    def fill_circle(self, x0, y0, r, color):
        """Draw a filled circle.

        Args:
            x0 (int): X coordinate of center point.
            y0 (int): Y coordinate of center point.
            r (int): Radius.
            color (int): RGB565 color value.
        """
        f = 1 - r
        dx = 1
        dy = -r - r
        x = 0
        y = r
        self.draw_vline(x0, y0 - r, 2 * r + 1, color)
        while x < y:
            if f >= 0:
                y -= 1
                dy += 2
                f += dy
            x += 1
            dx += 2
            f += dx
            self.draw_vline(x0 + x, y0 - y, 2 * y + 1, color)
            self.draw_vline(x0 - x, y0 - y, 2 * y + 1, color)
            self.draw_vline(x0 - y, y0 - x, 2 * x + 1, color)
            self.draw_vline(x0 + y, y0 - x, 2 * x + 1, color)

    def fill_ellipse(self, x0, y0, a, b, color):
        """Draw a filled ellipse.

        Args:
            x0, y0 (int): Coordinates of center point.
            a (int): Semi axis horizontal.
            b (int): Semi axis vertical.
            color (int): RGB565 color value.
        Note:
            The center point is the center of the x0,y0 pixel.
            Since pixels are not divisible, the axes are integer rounded
            up to complete on a full pixel.  Therefore the major and
            minor axes are increased by 1.
        """
        a2 = a * a
        b2 = b * b
        twoa2 = a2 + a2
        twob2 = b2 + b2
        x = 0
        y = b
        px = 0
        py = twoa2 * y
        # Plot initial points
        self.draw_line(x0, y0 - y, x0, y0 + y, color)
        # Region 1
        p = round(b2 - (a2 * b) + (0.25 * a2))
        while px < py:
            x += 1
            px += twob2
            if p < 0:
                p += b2 + px
            else:
                y -= 1
                py -= twoa2
                p += b2 + px - py
            self.draw_line(x0 + x, y0 - y, x0 + x, y0 + y, color)
            self.draw_line(x0 - x, y0 - y, x0 - x, y0 + y, color)
        # Region 2
        p = round(b2 * (x + 0.5) * (x + 0.5) +
                  a2 * (y - 1) * (y - 1) - a2 * b2)
        while y > 0:
            y -= 1
            py -= twoa2
            if p > 0:
                p += a2 - py
            else:
                x += 1
                px += twob2
                p += a2 - py + px
            self.draw_line(x0 + x, y0 - y, x0 + x, y0 + y, color)
            self.draw_line(x0 - x, y0 - y, x0 - x, y0 + y, color)

    def fill_hrect(self, x, y, w, h, color):
        """Draw a filled rectangle (optimized for horizontal drawing).

        Args:
            x (int): Starting X position.
            y (int): Starting Y position.
            w (int): Width of rectangle.
            h (int): Height of rectangle.
            color (int): RGB565 color value.
        """
        if self.is_off_grid(x, y, x + w - 1, y + h - 1):
            return
        chunk_height = 1024 // w
        chunk_count, remainder = divmod(h, chunk_height)
        chunk_size = chunk_height * w
        chunk_y = y
        if chunk_count:
            buf = color.to_bytes(2, 'big') * chunk_size
            for c in range(0, chunk_count):
                self.block(x, chunk_y,
                           x + w - 1, chunk_y + chunk_height - 1,
                           buf)
                chunk_y += chunk_height

        if remainder:
            buf = color.to_bytes(2, 'big') * remainder * w
            self.block(x, chunk_y,
                       x + w - 1, chunk_y + remainder - 1,
                       buf)

    def fill_rectangle(self, x, y, w, h, color):
        """Draw a filled rectangle.

        Args:
            x (int): Starting X position.
            y (int): Starting Y position.
            w (int): Width of rectangle.
            h (int): Height of rectangle.
            color (int): RGB565 color value.
        """
        if self.is_off_grid(x, y, x + w - 1, y + h - 1):
            return
        if w > h:
            self.fill_hrect(x, y, w, h, color)
        else:
            self.fill_vrect(x, y, w, h, color)

    def fill_polygon(self, sides, x0, y0, r, color, rotate=0):
        """Draw a filled n-sided regular polygon.

        Args:
            sides (int): Number of polygon sides.
            x0, y0 (int): Coordinates of center point.
            r (int): Radius.
            color (int): RGB565 color value.
            rotate (Optional float): Rotation in degrees relative to origin.
        Note:
            The center point is the center of the x0,y0 pixel.
            Since pixels are not divisible, the radius is integer rounded
            up to complete on a full pixel.  Therefore diameter = 2 x r + 1.
        """
        # Determine side coordinates
        coords = []
        theta = radians(rotate)
        n = sides + 1
        for s in range(n):
            t = 2.0 * pi * s / sides + theta
            coords.append([int(r * cos(t) + x0), int(r * sin(t) + y0)])
        # Starting point
        x1, y1 = coords[0]
        # Minimum Maximum X dict
        xdict = {y1: [x1, x1]}
        # Iterate through coordinates
        for row in coords[1:]:
            x2, y2 = row
            xprev, yprev = x2, y2
            # Calculate perimeter
            # Check for horizontal side
            if y1 == y2:
                if x1 > x2:
                    x1, x2 = x2, x1
                if y1 in xdict:
                    xdict[y1] = [min(x1, xdict[y1][0]), max(x2, xdict[y1][1])]
                else:
                    xdict[y1] = [x1, x2]
                x1, y1 = xprev, yprev
                continue
            # Non horizontal side
            # Changes in x, y
            dx = x2 - x1
            dy = y2 - y1
            # Determine how steep the line is
            is_steep = abs(dy) > abs(dx)
            # Rotate line
            if is_steep:
                x1, y1 = y1, x1
                x2, y2 = y2, x2
            # Swap start and end points if necessary
            if x1 > x2:
                x1, x2 = x2, x1
                y1, y2 = y2, y1
            # Recalculate differentials
            dx = x2 - x1
            dy = y2 - y1
            # Calculate error
            error = dx >> 1
            ystep = 1 if y1 < y2 else -1
            y = y1
            # Calcualte minimum and maximum x values
            for x in range(x1, x2 + 1):
                if is_steep:
                    if x in xdict:
                        xdict[x] = [min(y, xdict[x][0]), max(y, xdict[x][1])]
                    else:
                        xdict[x] = [y, y]
                else:
                    if y in xdict:
                        xdict[y] = [min(x, xdict[y][0]), max(x, xdict[y][1])]
                    else:
                        xdict[y] = [x, x]
                error -= abs(dy)
                if error < 0:
                    y += ystep
                    error += dx
            x1, y1 = xprev, yprev
        # Fill polygon
        for y, x in xdict.items():
            self.draw_hline(x[0], y, x[1] - x[0] + 2, color)

    def fill_vrect(self, x, y, w, h, color):
        """Draw a filled rectangle (optimized for vertical drawing).

        Args:
            x (int): Starting X position.
            y (int): Starting Y position.
            w (int): Width of rectangle.
            h (int): Height of rectangle.
            color (int): RGB565 color value.
        """
        if self.is_off_grid(x, y, x + w - 1, y + h - 1):
            return
        chunk_width = 1024 // h
        chunk_count, remainder = divmod(w, chunk_width)
        chunk_size = chunk_width * h
        chunk_x = x
        if chunk_count:
            buf = color.to_bytes(2, 'big') * chunk_size
            for c in range(0, chunk_count):
                self.block(chunk_x, y,
                           chunk_x + chunk_width - 1, y + h - 1,
                           buf)
                chunk_x += chunk_width

        if remainder:
            buf = color.to_bytes(2, 'big') * remainder * h
            self.block(chunk_x, y,
                       chunk_x + remainder - 1, y + h - 1,
                       buf)

    def is_off_grid(self, xmin, ymin, xmax, ymax):
        """Check if coordinates extend past display boundaries.

        Args:
            xmin (int): Minimum horizontal pixel.
            ymin (int): Minimum vertical pixel.
            xmax (int): Maximum horizontal pixel.
            ymax (int): Maximum vertical pixel.
        Returns:
            boolean: False = Coordinates OK, True = Error.
        """
        if xmin < 0:
            print('x-coordinate: {0} below minimum of 0.'.format(xmin))
            return True
        if ymin < 0:
            print('y-coordinate: {0} below minimum of 0.'.format(ymin))
            return True
        if xmax >= self.width:
            print('x-coordinate: {0} above maximum of {1}.'.format(
                xmax, self.width - 1))
            return True
        if ymax >= self.height:
            print('y-coordinate: {0} above maximum of {1}.'.format(
                ymax, self.height - 1))
            return True
        return False

    def load_sprite(self, path, w, h):
        """Load sprite image.

        Args:
            path (string): Image file path.
            w (int): Width of image.
            h (int): Height of image.
        Notes:
            w x h cannot exceed 2048
        """
        buf_size = w * h * 2
        with open(path, "rb") as f:
            return f.read(buf_size)

    def reset(self):
        """Perform reset: Low=initialization, High=normal operation."""
        self.rst(0)
        sleep_ms(50)
        self.rst(1)
        sleep_ms(50)

    def write_cmd(self, command, *args):
        """Write command to OLED.

        Args:
            command (byte): SSD1351 command code.
            *args (optional bytes): Data to transmit.
        """
        self.dc(0)
        self.cs(0)
        self.spi.write(bytearray([command]))
        self.cs(1)
        # Handle any passed data
        if len(args) > 0:
            self.write_data(bytearray(args))

    def write_data(self, data):
        """Write data to OLED.

        Args:
            data (bytes): Data to transmit.
        """
        self.dc(1)
        self.cs(0)
        self.spi.write(data)
        self.cs(1)

In [3]:
from machine import Pin, SPI, ADC
from time import sleep
def test_huzzah():
    """Test code."""
    for _ in range(20):
        spi = SPI(2, baudrate=14500000, sck=Pin(18), mosi=Pin(23))
        # Baud rate of 14500000 seems about the max
        print('spi started')
        display = Display(spi, dc=Pin(26), cs=Pin(5), rst=Pin(4))
        print('display started')

        display.clear(color565(0, 0, 0))
        sleep(1)
        for i in range(127):
            val = int(adc.read())
            display.draw_vline(i%127, 0, min(max(int(val/15)-44,10),127), color565((i%255), 255-(i%255), 255-int(((i%255)/2))))
            sleep(0.01)
        display.clear()
        display.cleanup()

In [6]:
test_huzzah()

Traceback (most recent call last):
  File "<stdin>", line 1, in <module>
  File "<stdin>", line 17, in test_huzzah
NameError: name 'adc' is not defined


In [5]:
import sys
sys.exit()

In [ ]:
from machine import Pin, SPI, ADC
from time import sleep
spi = SPI(2, 14500000, miso=Pin(19), mosi=Pin(18), sck=Pin(5)) # Refresh rate is faster on SSD1351
#spi = SPI(2, baudrate=14500000, sck=Pin(18), mosi=Pin(23))
#spi = SPI(-1, 10000000, miso=Pin(19), mosi=Pin(18), sck=Pin(5)) # WORKS ON SSD1351 AND MAX7219 (8x8 matrix display)


In [8]:
spi.deinit()

Traceback (most recent call last):
  File "<stdin>", line 1, in <module>
NameError: name 'spi' is not defined


In [4]:
from ssd1351go import Display, color565
display = Display(spi, dc=Pin(25), cs=Pin(4), rst=Pin(26))

Traceback (most recent call last):
  File "<stdin>", line 2, in <module>
NameError: name 'spi' is not defined


In [102]:
from ssd1351 import Display, color565
display = Display(spi, rst=Pin(26), dc=Pin(25), cs=Pin(4))

Traceback (most recent call last):
  File "<stdin>", line 2, in <module>
NameError: name 'spi' is not defined


In [103]:
display = Display(spi, dc=Pin(25), cs=Pin(4), rst=Pin(26))

Traceback (most recent call last):
  File "<stdin>", line 1, in <module>
NameError: name 'spi' is not defined


In [ ]:
from machine import Pin, PWM
l = PWM(Pin(25), freq=50, duty=300)
r = PWM(Pin(26), freq=50, duty=300)

In [5]:
display.clear(color565(64, 0, 255))

sleep(1)

display.clear()
for i in range(127):
    val = 100
    display.draw_vline(i%127, 0, min(max(int(val/15)-44,10),127), color565((i%255), 255-(i%255), 255-int(((i%255)/2))))
    sleep(0.03)

In [1]:
from machine import Pin, PWM
sound = PWM(Pin(13), freq=330, duty=0)

# IRQ

In [ ]:
from machine import Pin, SPI, ADC
from time import sleep
mic = ADC(Pin(33))
mic.atten(ADC.ATTN_11DB)
print(mic.read())

In [74]:
import micropython
micropython.alloc_emergency_exception_buf(100)

In [2]:
joystick_sw = Pin(27, Pin.IN ,Pin.PULL_UP)
print(joystick_sw.value())
def b(p):
    print("sound ", p.value())
    if p.value() == 1:
        sound.duty(0)
    else:
        sound.duty(1000)
            
joystick_sw.irq(b, Pin.IRQ_RISING)

1


In [58]:
display.clear(color565(0, 0, 0))
sleep(0.5)
try:
    while True:
        for i in range(127):
            val = int(mic.read())
            display.draw_vline(i%127, 0, min(max(int(val/15)-44,10),127), color565((i%255), 255-(i%255), 255-int(((i%255)/2))))
            sleep(0.01)
        display.clear(color565(0, 0, 0))
        gc.collect()
except KeyboardInterrupt:
    display.clear(color565(0, 0, 0))
    gc.collect()

In [ ]:
import gc
gc.collect()

In [13]:
display.clear(color565(0, 0, 0))

In [15]:
from xglcd_font import XglcdFont
display.clear(color565(0, 0, 0))
#broadway = XglcdFont('fonts/Broadway17x15.c', 17, 15)
#espresso_dolce = XglcdFont('fonts/EspressoDolce18x24.c', 18, 24)
#arcadepix = XglcdFont('fonts/ArcadePix9x11.c', 9, 11)
font_b = XglcdFont('fonts/Bally7x9.c', 7, 9)
#wendy = XglcdFont('fonts/Wendy7x8.c', 7, 8)
ast = ["def kebab():", "  var ostepop = 1", "    return ostepop"]
for i, a in enumerate(ast):
    print(a)
    display.draw_text(0, 60+(i*10), a, font_b, color565(255, 255, 0))
    sleep(2)
#display.clear(color565(0, 0, 0))

def kebab():
  var ostepop = 1
    return ostepop


In [71]:
print('display started')
display.clear()
display.cleanup()

Traceback (most recent call last):
  File "<stdin>", line 2, in <module>
NameError: name 'display' is not defined


In [1]:
from machine import Pin, ADC
sw = Pin(27, Pin.IN, Pin.PULL_UP)
adcl = ADC(Pin(32))
adcl.atten(ADC.ATTN_11DB)
adcr = ADC(Pin(39))
adcr.atten(ADC.ATTN_11DB)

In [101]:
print(int(adcl.read()), int(adcr.read()), sw.value() == 1)

1328 2451 True


In [7]:
while True:
    if sw.value() == 0:
        display.clear(color565(0, 0, 0))
        break
    x = 127 - (max(3, min(124, int(adcl.read()/32))))
    y = max(3, min(124, int(adcr.read()/32)))
    display.clear(color565(0, 0, 0))
    display.draw_circle(x, y, 3, color565(0, 0, 255))
    sleep(0.1)
    

y-coordinate: 130 above maximum of 127.
y-coordinate: 130 above maximum of 127.
y-coordinate: 130 above maximum of 127.
y-coordinate: 128 above maximum of 127.
y-coordinate: 128 above maximum of 127.
y-coordinate: 129 above maximum of 127.
y-coordinate: 129 above maximum of 127.
y-coordinate: 129 above maximum of 127.
y-coordinate: 129 above maximum of 127.
y-coordinate: 130 above maximum of 127.
y-coordinate: 130 above maximum of 127.
y-coordinate: 130 above maximum of 127.
y-coordinate: 128 above maximum of 127.
y-coordinate: 128 above maximum of 127.
y-coordinate: 129 above maximum of 127.
y-coordinate: 129 above maximum of 127.
y-coordinate: 129 above maximum of 127.
y-coordinate: 129 above maximum of 127.
x-coordinate: 130 above maximum of 127.
x-coordinate: 128 above maximum of 127.
x-coordinate: 128 above maximum of 127.
x-coordinate: 130 above maximum of 127.
x-coordinate: 130 above maximum of 127.
x-coordinate: 129 above maximum of 127.
x-coordinate: 129 above maximum of 127.


In [22]:
def test():
    """Test code."""
    for _ in range(20):
        spi = SPI(2, baudrate=14500000, sck=Pin(18), mosi=Pin(23))
        # Baud rate of 14500000 seems about the max
        print('spi started')
        display = Display(spi, dc=Pin(17), cs=Pin(5), rst=Pin(16))
        print('display started')

        display.clear(color565(0, 0, 0))
        sleep(1)

    
        for i in range(127):
            val = int(adc.read())
            display.draw_vline(i%127, 0, min(max(int(val/15)-44,10),127), color565((i%255), 255-(i%255), 255-int(((i%255)/2))))
            sleep(0.01)
        display.clear()
        display.cleanup()

In [23]:
test()

spi started
display started
display off
spi started
display started
display off
spi started
display started
display off
spi started
display started
display off
spi started
display started
display off
spi started
display started
display off
spi started
display started
display off
spi started
display started
display off
spi started
display started
display off
spi started
display started
display off
spi started
display started
display off
spi started
display started
display off
spi started
display started
display off
spi started
display started
display off
spi started
display started
display off
spi started
display started
display off
spi started
display started
display off
spi started
display started
display off
spi started
display started
display off
spi started
display started
display off


# Shapes

In [ ]:
"""SSD1351 demo (shapes)."""
from time import sleep
#from ssd1351 import Display, color565
from machine import Pin, SPI


def test():
    """Test code."""
    # Baud rate of 14500000 seems about the max
    spi = SPI(2, baudrate=14500000, sck=Pin(18), mosi=Pin(23))
    print('spi started')
    display = Display(spi, dc=Pin(17), cs=Pin(5), rst=Pin(16))
    print('display started')

    display.clear(color565(64, 0, 255))
    sleep(1)

    display.clear()

    display.draw_hline(10, 127, 63, color565(255, 0, 255))
    sleep(1)

    display.draw_vline(10, 0, 127, color565(0, 255, 255))
    sleep(1)

    display.fill_hrect(23, 50, 30, 75, color565(255, 255, 255))
    sleep(1)

    display.draw_hline(0, 0, 127, color565(255, 0, 0))
    sleep(1)

    display.draw_line(127, 0, 64, 127, color565(255, 255, 0))
    sleep(2)

    display.clear()

    coords = [[0, 63], [78, 80], [122, 92], [50, 50], [78, 15], [0, 63]]
    display.draw_lines(coords, color565(0, 255, 255))
    sleep(1)

    display.clear()
    display.fill_polygon(7, 63, 63, 50, color565(0, 255, 0))
    sleep(1)

    display.fill_rectangle(0, 0, 15, 127, color565(255, 0, 0))
    sleep(1)

    display.clear()

    display.fill_rectangle(0, 0, 63, 63, color565(128, 128, 255))
    sleep(1)

    display.draw_rectangle(0, 64, 63, 63, color565(255, 0, 255))
    sleep(1)

    display.fill_rectangle(64, 0, 63, 63, color565(128, 0, 255))
    sleep(1)

    display.draw_polygon(3, 96, 96, 30, color565(0, 64, 255),
                         rotate=15)
    sleep(3)

    display.clear()

    display.fill_circle(32, 32, 30, color565(0, 255, 0))
    sleep(1)

    display.draw_circle(32, 96, 30, color565(0, 0, 255))
    sleep(1)

    display.fill_ellipse(96, 32, 30, 16, color565(255, 0, 0))
    sleep(1)

    display.draw_ellipse(96, 96, 16, 30, color565(255, 255, 0))

    sleep(5)
    display.cleanup()


test()

# Boxes

In [ ]:
"""SSD1351 demo (bouncing boxes)."""
from machine import Pin, SPI
from random import random, seed
#from ssd1351 import Display, color565
from utime import sleep_us, ticks_cpu, ticks_us, ticks_diff


class Box(object):
    """Bouncing box."""

    def __init__(self, screen_width, screen_height, size, display, color):
        """Initialize box.
        Args:
            screen_width (int): Width of screen.
            screen_height (int): Width of height.
            size (int): Square side length.
            display (SSD1351): OLED display object.
            color (int): RGB565 color value.
        """
        self.size = size
        self.w = screen_width
        self.h = screen_height
        self.display = display
        self.color = color
        # Generate non-zero random speeds between -5.0 and 5.0
        seed(ticks_cpu())
        r = random() * 10.0
        self.x_speed = 5.0 - r if r < 5.0 else r - 10.0
        r = random() * 10.0
        self.y_speed = 5.0 - r if r < 5.0 else r - 10.0

        self.x = self.w / 2.0
        self.y = self.h / 2.0
        self.prev_x = self.x
        self.prev_y = self.y

    def update_pos(self):
        """Update box position and speed."""
        x = self.x
        y = self.y
        size = self.size
        w = self.w
        h = self.h
        x_speed = abs(self.x_speed)
        y_speed = abs(self.y_speed)
        self.prev_x = x
        self.prev_y = y

        if x + size >= w - x_speed:
            self.x_speed = -x_speed
        elif x - size <= x_speed + 1:
            self.x_speed = x_speed

        if y + size >= h - y_speed:
            self.y_speed = -y_speed
        elif y - size <= y_speed + 1:
            self.y_speed = y_speed

        self.x = x + self.x_speed
        self.y = y + self.y_speed

    def draw(self):
        """Draw box."""
        x = int(self.x)
        y = int(self.y)
        size = self.size
        prev_x = int(self.prev_x)
        prev_y = int(self.prev_y)
        self.display.fill_hrect(prev_x - size,
                                prev_y - size,
                                size, size, 0)
        self.display.fill_hrect(x - size,
                                y - size,
                                size, size, self.color)


def test():
    """Bouncing box."""
    try:
        # Baud rate of 14500000 seems about the max
        spi = SPI(2, baudrate=14500000, sck=Pin(18), mosi=Pin(23))
        display = Display(spi, dc=Pin(17), cs=Pin(5), rst=Pin(16))
        display.clear()

        colors = [color565(255, 0, 0),
                  color565(0, 255, 0),
                  color565(0, 0, 255),
                  color565(255, 255, 0),
                  color565(0, 255, 255),
                  color565(255, 0, 255)]
        sizes = [12, 11, 10, 9, 8, 7]
        boxes = [Box(128, 128, sizes[i], display,
                 colors[i]) for i in range(6)]

        while True:
            timer = ticks_us()
            for b in boxes:
                b.update_pos()
                b.draw()
            # Attempt to set framerate to 30 FPS
            timer_dif = 33333 - ticks_diff(ticks_us(), timer)
            if timer_dif > 0:
                sleep_us(timer_dif)

    except KeyboardInterrupt:
        display.cleanup()


test()

# Palette

In [1]:
"""SSD1351 demo (color palette)."""
from time import sleep
#from ssd1351 import Display, color565
from machine import Pin, SPI


def hsv_to_rgb(h, s, v):
    """
    Convert HSV to RGB (based on colorsys.py).

        Args:
            h (float): Hue 0 to 1.
            s (float): Saturation 0 to 1.
            v (float): Value 0 to 1 (Brightness).
    """
    if s == 0.0:
        return v, v, v
    i = int(h * 6.0)
    f = (h * 6.0) - i
    p = v * (1.0 - s)
    q = v * (1.0 - s * f)
    t = v * (1.0 - s * (1.0 - f))
    i = i % 6

    v = int(v * 255)
    t = int(t * 255)
    p = int(p * 255)
    q = int(q * 255)

    if i == 0:
        return v, t, p
    if i == 1:
        return q, v, p
    if i == 2:
        return p, v, t
    if i == 3:
        return p, q, v
    if i == 4:
        return t, p, v
    if i == 5:
        return v, p, q


def test():
    """Test code."""
    # Baud rate of 14500000 seems about the max
    spi = SPI(2, baudrate=14500000, sck=Pin(18), mosi=Pin(23))
    display = Display(spi, dc=Pin(17), cs=Pin(5), rst=Pin(16))

    c = 0
    for x in range(0, 128, 16):
        for y in range(0, 128, 16):
            color = color565(*hsv_to_rgb(c / 64, 1, 1))
            display.fill_circle(x + 7, y + 7, 7, color)
            c += 1
    sleep(9)
    display.cleanup()


test()

Traceback (most recent call last):
  File "<stdin>", line 60, in <module>
  File "<stdin>", line 47, in test
OSError: SPI device already in use
